Three principle types of services. __Cluster IP, node port and load balancer__. These services build conceptually on one another

__Cluster IP__ service has a static IP address, and operates as a traffic distributor within the cluster. But aren't accessible by resources outside the cluster

```
apiVersion: 1
kind: Service
metadata:
    name: my-service
spec:
    type: ClusterIP # default ClusterIP
    selector:
        # pods with label app:backend selected and included as endpoints for this service
        # always create a service before creating any workloads that need to access this service
        # so that the pods have host name and IP address of the service in their env variable
        app: Backend
    ports:
        - protocol: TCP
          port: 80 # receive on port 80 and remap it to 9376
          targetPort: 9376 # port of container
```

__Node port__ is for external communications, it built on top of cluster IP, so when create a node port, a cluster IP service is created automatically. Can be useful to expose a service through an external load balancer.   
You would have to deal with node management, make sure there is no port collusions

```
apiVersion: 1
kind: Service
metadata:
    name: my-service
spec:
    type: NodePort # default ClusterIP
    selector:
        # pods with label app:backend selected and included as endpoints for this service
        # always create a service before creating any workloads that need to access this service
        # so that the pods have host name and IP address of the service in their env variable
        app: Backend
    ports:
        - protocol: TCP
          nodePort: 30100 # range 30,000 to 32767, but this is usually automatically set
          port: 80 # receive on port 80 and remap it to 9376
          targetPort: 9376 # port of container
```

__load balancer__ builds on the cluster IP and can be used to expose a servie to resources outside of the cluster.
With GKE, the loadBalancer service is implemented using GDP's network loadBalancer

```
apiVersion: 1
kind: Service
metadata:
    name: my-service
spec:
    type: LoadBalancer # GCP will assign a static LoadBalancer IP address accessible from outside.
    # for lowest possible latency, 
    # force Q proxy to choose a pod local to the node that receives the client traffic
    externalTrafficPolicy: Local
    selector:
        app: Backend
    ports:
        - protocol: TCP
          port: 80 # receive on port 80 and remap it to 9376
          targetPort: 9376 # port of container
```

tradeoff between lowest possible latency or the more even cluster load balancing

If we want to have connectivity to our pods, we will need to create a service. In Kubernetes, a service is a network abstraction over a set of pods. This allows for the traffic to be load balanced for failures. A service allows Kubernetes to set a single DNS record for the pods. As we mentioned earlier, each pod has a separate IP address.

Expose the Pod for testing
--

create a service to expose our nginx Pod externally
```
kubectl expose pod $my_nginx_pod --port 80 --type LoadBalancer
```

view details about services in the cluster
```
kubectl get services
```

verify that the nginx container is serving the static HTML file that you copied.

You replace [EXTERNAL_IP] with the external IP address of your service that you obtained from the output of the previous step.
```
curl http://[EXTERNAL_IP]/test.html
```

view the resources being used by the nginx Pod
```
kubectl top pods
```

Blue-green deployments
--

```
kubectl apply -f my-app-v2.yaml
```
When a new deployment labeled v2 in this case is created and is ready, the version label on the service is changed to the newer version, labeled v2 in this example. Now, the traffic will be directed to the nearest set of pods. The green deployment with the v2 version label. Set up the old blue deployment pods that have the v1 version label. The blue deployment with the older version can then be deleted.

Canary deployments
--

the new version of the deployment will start with zero replicas running. Over time as a new version is scaled up, the old version of the deployment can be scaled down, and eventually deleted.

```
kubectl apply -f my-app-v2.yaml

kubectl scale deploy/my-app-v2 -replicas=10

kubectl delete -f my-app-v2.yaml

```